In [1]:
import torch
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

/home/matti/code/Apps/TextualMediaSearcher/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Embeddings models


#### BERT (AutoModel)

In [2]:
# Finnish language embedding model
model_name = "TurkuNLP/bert-base-finnish-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_sentences(sents: list[str] | str) -> np.ndarray:
    if not isinstance(sents, list):
        sents = [sents]
    inputs = tokenizer(sents, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings.cpu().numpy()

In [3]:
# Example usage
text = "Koirilla on pentuja."
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1) # sentence embedding (shape: (batch, hidden_size))
embeddings.shape

torch.Size([1, 768])

#### Sentence Transformers

In [2]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

def embed_sentences(sents: list[str] | str) -> np.ndarray:
    """  """
    if not isinstance(sents, list):
        sents = [sents]
    return model.encode(sents)

## Main

In [4]:
# Load example text
text = ""
with open('suomi_text.txt', 'r') as f:
    for line in f:
        if line != "\n":
            text += line.replace("\n", "").strip() + " "

In [14]:
# get sentence embeddings
sents = sent_tokenize(text)
batch_size = 16

embeddings_size = model.get_sentence_embedding_dimension()
sent_embeddings = np.zeros((len(sents), embeddings_size), dtype="float32")
for i in range(0, len(sents), batch_size):
    print(" {:>3}/{}".format(i, len(sents)))
    batch = sents[i : i+batch_size]
    batch_embeds = embed_sentences(batch)
    sent_embeddings[i:i+batch_size, :] = batch_embeds

   0/159
  16/159
  32/159
  48/159
  64/159
  80/159
  96/159
 112/159
 128/159
 144/159


#### Save & Load

In [11]:
# Save embeddings
import pickle
id2idx = {}
data = {
    "embeddings": sent_embeddings,
    "id2idx": id2idx,
}
with open("sent_embeddings.pkl", "wb") as f:
    pickle.dump(data, f)

In [12]:
# Load embeddings
with open("sent_embeddings.pkl", "rb") as f:
    data = pickle.load(f)
sent_embeddings = data['embeddings']
id2idx = data['id2idx']

#### Run

In [15]:
# SIMILARITY
text = "Suomi oli venäjän vallan alla"
query_emb = embed_sentences(text)
sims = cosine_similarity(sent_embeddings, query_emb).squeeze()
sorted_idx = np.flip( np.argsort(sims, axis=0) )[:100].squeeze()
pairs = np.array((sorted_idx, sims[sorted_idx])).T
for idx, sim in pairs[:5]:
    sent = sents[int(idx)]
    print(f"{sim:.4f}  |  {sent}")

0.6892  |  Suomen asema hyökkäyksessä oli pitkään epäselvä, mutta sille oli joka tapauksessa varattu siinä rooli.
0.6491  |  Venäjän vallan aika oli Suomen kannalta yksi historian rauhallisimmista kausista (vain Oolannin sota kosketti Suomen aluetta), mutta jonkin verran suomalaisia vapaaehtoisjoukkoja tai upseereita taisteli Venäjän lipun alla Manner-Euroopassa ja Aasiassa.
0.6450  |  Suomen asema välirauhan aikaan talvisodan jälkeen oli ollut hyvin tukala ja poliittinen liikkumavara oli niukka.
0.6267  |  Nyt Suomi nähtiin tärkeänä osana hyökkäyksessä itään.
0.6215  |  Alkuvaiheessa kuitenkin talonpoikainen nostoväki oli edelleen sotavoimien ydinjoukkona.
